# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [18]:
import os
import glob
import psycopg2
import pandas as pd
import json

In [19]:
from sql_queries import *

In [20]:
with open("../config.json") as f:
    config = json.load(f)
    
conn = psycopg2.connect(host = config["host"],
                        dbname = config["dbname"],
                        user = config["username"],
                        password = config["password"])

conn.set_session(autocommit=True)
cur = conn.cursor()

In [21]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

## Due to our Foreign key constraints, we will have to fill parent tables first, before filling child tables.
For that, we can fill the tables in the following order:
1. artist table (dimension table)
2. Song table (dimension table)
3. User table (dimension table)
4. Time Table (dimension table)
5. song played table (fact table)

## Process Artist Data

### Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [22]:
song_files = get_files("../data/song_data/")
song_files[:5]

['c:\\MyStuff\\Data Engineer FWD\\Project 1 Postgres ETL\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json',
 'c:\\MyStuff\\Data Engineer FWD\\Project 1 Postgres ETL\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json',
 'c:\\MyStuff\\Data Engineer FWD\\Project 1 Postgres ETL\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json',
 'c:\\MyStuff\\Data Engineer FWD\\Project 1 Postgres ETL\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json',
 'c:\\MyStuff\\Data Engineer FWD\\Project 1 Postgres ETL\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json']

In [23]:
filepath = song_files[0]

In [24]:
df = pd.read_json(filepath, lines = True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


### Artist table

In [13]:
columns = ["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]
artist_data = df.loc[0, columns]
artist_data = list(artist_data.values)
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

In [12]:
cur.execute(artist_table_insert, artist_data)

InvalidTextRepresentation: invalid input syntax for type double precision: "California - LA"
LINE 3: ...         VALUES('ARD7TVE1187B99BFB1', Null, Null, 'Californi...
                                                             ^


We need a way to map numpy nans to Nulls in the database. Using Pycopg2, we can do the following

In [11]:
import numpy as np
from psycopg2.extensions import AsIs, register_adapter

def adapt_npfloat_and_nans(np_float,
                           _FLOAT = psycopg2.extensions.Float,
                           _NULL = psycopg2.extensions.AsIs('Null')):
    if np.isnan(np_float):
        return _NULL
    return _FLOAT(np_float)

register_adapter(np.float64, adapt_npfloat_and_nans)

In [14]:
cur.execute(artist_table_insert, artist_data)

We can do a sanity check to see if it works

In [16]:
cur.execute("SELECT * FROM artist")
res = cur.fetchone()
while res:
    print(res)
    res = cur.fetchone() 

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None)


## #2: `songs` Table

In [17]:
columns = ["song_id", "title", "artist_id", "year", "duration"]
song_data = df.loc[0, columns]
song_data = list(song_data.values)
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)

ProgrammingError: can't adapt type 'numpy.int64'

This is an error that occurs because the database driver doesn't know what to do with np datatypes. A proposed solution on [stackoverflow](https://stackoverflow.com/questions/50626058/psycopg2-cant-adapt-type-numpy-int64) shows that we can use the string representation of those datatypes as proxy for their value, because their string representation shouldn't differ from their value.

This is done via the `AsIs` extension method in psycopg2, and regsitering the string representation such that the driver does that representation whenever it encounters a value of that type

Another solution [Stackoverflow](https://stackoverflow.com/questions/32107558/how-do-i-convert-numpy-nan-objects-to-sql-nulls)

In [11]:
import numpy as np
from psycopg2.extensions import AsIs, register_adapter
def adapt_np_int(np_int:np.int64):
    _INT = psycopg2.extensions.INTEGER
    return _INT(np_int)
    
register_adapter(np.int64, adapt_np_int)

In [17]:
x = psycopg2.extensions.INTEGER
x(232)

TypeError: function takes exactly 2 arguments (1 given)

As for floats, we need to take care of the `nan` values and set them to NULL in the database. Otherwise, we will revert them to the default float

In [10]:
def adapt_npfloat_and_nans(np_float,
                           _FLOAT = psycopg2.extensions.Float,
                           _NULL = psycopg2.extensions.AsIs('Null')):
    if np.isnan(np_float):
        return _NULL
    return _FLOAT(np_float)

register_adapter(np.float64, adapt_npfloat_and_nans)

NameError: name 'register_adapter' is not defined

In [13]:
cur.execute(song_table_insert, song_data)

TypeError: function takes exactly 2 arguments (1 given)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
columns = ["artist_id", "artist_latitude", "artist_longitude", "artist_location", "artist_name"]
artist_data = df.loc[0, columns]
artist_data = list(artist_data.values)
artist_data

['ARD7TVE1187B99BFB1', nan, nan, 'California - LA', 'Casual']

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)

InvalidTextRepresentation: invalid input syntax for type double precision: "California - LA"
LINE 3: ...'ARD7TVE1187B99BFB1', 'NaN'::float, 'NaN'::float, 'Californi...
                                                             ^


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = 

In [ ]:
filepath = 

In [ ]:
df = 
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = 
df.head()

In [ ]:
t = 
t.head()

In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.